In [1]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.svm import SVC
from keras.utils.np_utils import to_categorical
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [2]:
(img_rows, img_cols, nb_channels) = (299, 299, 3)
image_shape = (299, 299, 3)

def read_data(path='../data/train.csv'):  
    data_train = pd.read_csv(path)
    X_data_train = data_train['ImageId']
    y_data_train = data_train['Malignant']
    X_train_id = X_data_train.values
    Y_train = y_data_train.values

    # input image dimensions
    (img_rows, img_cols, nb_channels) = (299, 299, 3)
    image_shape = (299, 299, 3)
    data_size = X_train_id.shape[0]

    # read data
    X_image = np.zeros((data_size,img_rows, img_cols, nb_channels))
    #X_image_mask = np.zeros((data_size,img_rows, img_cols, nb_channels))

    for i in range(data_size):
        name = X_train_id[i]
        filename = '../data/im/{}.jpg'.format(name)
        image = imread(filename)
        #filename_Segmentation = 'data/im/{}_segmentation.jpg'.format(name)
        #image_Segmentation = imread(filename_Segmentation) # Value 0 or 255
        #image_Segmentation_boolean = (image_Segmentation/255).astype(np.uint8) # To get uint8
        #image_Segmentation_expand = np.expand_dims(image_Segmentation_boolean, axis=2)
        #image_mul_mask = (image_Segmentation_expand*image) 
        image = resize(image,image_shape, mode='reflect')
        #image_mul_mask = resize(image_mul_mask,image_shape, mode='reflect')
        X_image[i]= image
        #X_image_mask[i] = image_mul_mask
    #X_image_mask = np.moveaxis(X_image_mask, -1, 1)
    return X_image.astype('float32')/255,Y_train[:data_size]

In [3]:
data,labels = read_data()
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, shuffle= True)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

E:\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [4]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print('There is {} train data'.format(X_train.shape[0]))
print('There is {} test data'.format(X_test.shape[0]))
print(img_rows, img_cols, nb_channels)

X_train shape: (630, 299, 299, 3)
y_train shape: (630,)
X_test shape: (70, 299, 299, 3)
y_test shape: (70,)
There is 630 train data
There is 70 test data
299 299 3


In [5]:
traingen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

validgen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
traingen.fit(X_train)
validgen.fit(X_test)

In [8]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

x = layers.Dropout(0.2)(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(traingen.flow(X_train, y_train, batch_size=16),
      steps_per_epoch=len(X_train) / 16 ,
      epochs=10,
      validation_data=validgen.flow(X_test, y_test, batch_size=16),
      validation_steps=len(X_test) / 16,
      class_weight=class_weights,
      verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(traingen.flow(X_train, y_train, batch_size=16),
      steps_per_epoch=len(X_train) / 16 ,
      epochs=20,
      validation_data=validgen.flow(X_test, y_test, batch_size=16),
      validation_steps=len(X_test) / 16,
      class_weight=class_weights,
      verbose=1)

# Save the model
model.save('InterceptionV3.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
40/39 [==============================] - 350s 9s/step - loss: 5.3731 - val_loss: 2.9315
Epoch 2/10
40/39 [==============================] - 337s 8s/step - loss: 0.9252 - val_loss: 1.0310
Epoch 3/10
40/39 [==============================] - 337s 8s/step - loss: 0.7510 - val_loss: 0.8076
Epoch 4/10
40/39 [==============================] - 336s 8s/step - loss: 0.6720 - val_loss: 0.7007
Epoch 5/10
40/39 [==============================] - 336s 8s/step - loss: 0.6270 - val_loss: 1.2669
Epoch 6/10
40/39 [==============================] - 335s 8s/step - loss: 0.6433 - val_loss: 0.7160
Epoch 7/10
40/39 [==============================] - 336s 8s/step - loss: 0.6685 - val_loss: 0.9636
Epoch 8/10
40/39 [==============================] - 336s 8s/step - loss: 0.6036 - val_loss: 0.8377
Epoch 9/10
40/39 [===========================

40/39 [==============================] - 411s 10s/step - loss: 0.4972 - val_loss: 0.7134
Epoch 6/20
40/39 [==============================] - 413s 10s/step - loss: 0.5094 - val_loss: 0.7054
Epoch 7/20
40/39 [==============================] - 411s 10s/step - loss: 0.4892 - val_loss: 0.6765
Epoch 8/20
40/39 [==============================] - 410s 10s/step - loss: 0.4989 - val_loss: 0.6995
Epoch 9/20
40/39 [==============================] - 408s 10s/step - loss: 0.4645 - val_loss: 0.7001
Epoch 10/20
40/39 [==============================] - 408s 10s/step - loss: 0.4752 - val_loss: 0.7023
Epoch 11/20
40/39 [==============================] - 409s 10s/step - loss: 0.4847 - val_loss: 0.7475
Epoch 12/20
40/39 [==============================] - 408s 10s/step - loss: 0.4768 - val_loss: 0.6726
Epoch 13/20
40/39 [==============================] - 408s 10s/step - loss: 0.4647 - val_loss: 0.7327
Epoch 14/20
40/39 [==============================] - 408s 10s/step - loss: 0.4686 - val_loss: 0.7383
Epoch 